# Annotate an h5ad file based on CELLxGENE schema

This guide shows how to validate and curate an AnnData object using the metadata registries of [laminlabs/cellxgene](https://lamin.ai/laminlabs/cellxgene), based on the [CELLxGENE schema version 5.1.0](https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/5.1.0/schema.md).

The validated object can be subsequently registered as an artifact in your LaminDB instance.

```{note}

The Annotate class is primarily designed to validate all metadata with respect to adhere to the ontologies.
It not reimplement all rules of the cellxgene schema and we therefore recommend running the [cellxgene-schema](https://github.com/chanzuckerberg/single-cell-curation) if full adherence beyond metadata is a necessity.
```

## Set up

Load your instance to register the annotated AnnData:

In [ ]:
!lamin init --storage ./test-cellxgene-annotate --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
from cellxgene_lamin import Annotate, datasets, CellxGeneFields

ln.settings.verbosity = "hint"
lb.settings.organism = "human"

## An h5ad file

Let's start with an AnnData object that we'd like to inspect and curate:

In [ ]:
adata = datasets.anndata_human_immune_cells(populate_registries=True)
adata

In [ ]:
adata.write_h5ad("anndata_human_immune_cells.h5ad")

In [ ]:
!cellxgene-schema validate anndata_human_immune_cells.h5ad

## Validate and curate metadata

Validate the AnnData object:

In [ ]:
try:
    annotate = Annotate(adata)
except Exception as e:
    print(e)

Let's fix the "donor_id" column name:

In [ ]:
adata.obs.rename(columns={"donor": "donor_id"}, inplace=True)

For the missing columns, we can pass default values suggested from CELLxGENE:

In [ ]:
CellxGeneFields.OBS_FIELD_DEFAULTS

In [ ]:
annotate = Annotate(adata, organism="human", **CellxGeneFields.OBS_FIELD_DEFAULTS)

In [ ]:
annotate.obs_fields

In [ ]:
validated = annotate.validate()

In [ ]:
validated

## Register new metadata labels

Following the suggestions above to register genes and labels that aren't present in the current instance:

(Note that our instance is rather empty. Once you filled up the registries, registering new labels won't be frequently needed)

In [ ]:
annotate.update_registry(feature="all")

For donors, we register the new labels:

In [ ]:
annotate.update_registry(feature="donor_id", validated_only=False)

An error is shown for the tissue label "lungg", which is a typo, should be "lung". Let's fix it:

In [ ]:
tissues = annotate.lookup()["tissue"]

In [ ]:
# using a lookup object to find the correct term
tissues.lung

In [ ]:
adata.obs["tissue"] = adata.obs["tissue"].cat.rename_categories(
    {"lungg": tissues.lung.name}
)

In [ ]:
annotate.update_registry("tissue")

Let's validate the object again:

In [ ]:
validated = annotate.validate()

In [ ]:
validated

In [ ]:
adata.obs.head()

## Register file

Now we are ready to register the artifact to the working instance:

In [ ]:
# track the current notebook
ln.transform.stem_uid = "WOK3vP0bNGLx"
ln.transform.version = "0"
ln.track()

In [ ]:
# this will modify the AnnData object by adding required columns and categories
artifact = annotate.register_artifact(description="test h5ad file")

View the registered artifact with metadata:

In [ ]:
artifact.describe()

## Register collection

Register a new collection for the registered artifact:

In [ ]:
# register a new collection
collection = annotate.register_collection(
    artifact,  # registered artifact above, can also pass a list of artifacts
    name=(  # title of the publication
        "Cross-tissue immune cell analysis reveals tissue-specific features in humans"
        " (for test demo only)"
    ),
    description="10.1126/science.abl5197",  # DOI of the publication
    reference="E-MTAB-11536",  # accession number (e.g. GSE#, E-MTAB#, etc.)
    reference_type="ArrayExpress",
)  # source type (e.g. GEO, ArrayExpress, SRA, etc.)

In [ ]:
collection.artifact

In [ ]:
artifact.collection

## Return an input h5ad file for cellxgene-schema

In [ ]:
adata_cxg = annotate.to_cellxgene(is_primary_data=True)
adata_cxg

In [ ]:
adata_cxg.write_h5ad("anndata_human_immune_cells_cxg.h5ad")

In [ ]:
!cellxgene-schema validate anndata_human_immune_cells_cxg.h5ad